<a href="https://colab.research.google.com/github/amimulhasan/Deep-Learning/blob/main/VIT_GRU_Lung_Cancer.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json

# Download the lung disease dataset
!kaggle datasets download fatemehmehrparvar/lung-disease
!unzip -q lung-disease.zip


Dataset URL: https://www.kaggle.com/datasets/fatemehmehrparvar/lung-disease
License(s): Attribution 4.0 International (CC BY 4.0)
  0% 0.00/34.9M [00:00<?, ?B/s]
100% 34.9M/34.9M [00:00<00:00, 1.10GB/s]


In [ ]:
import tensorflow as tf
from tensorflow.keras.layers import Dense, Dropout, LayerNormalization, GRU, Input
from tensorflow.keras.models import Model
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import matplotlib.pyplot as plt


In [ ]:
img_size = 128
batch_size = 32
data_dir = "/content/Lung X-Ray Image/Lung X-Ray Image"

datagen = ImageDataGenerator(
    rescale=1./255,
    validation_split=0.2,
    zoom_range=0.2,
    rotation_range=20,
    horizontal_flip=True
)

train_data = datagen.flow_from_directory(
    data_dir,
    target_size=(img_size, img_size),
    batch_size=batch_size,
    class_mode='categorical',
    subset='training'
)

val_data = datagen.flow_from_directory(
    data_dir,
    target_size=(img_size, img_size),
    batch_size=batch_size,
    class_mode='categorical',
    subset='validation'
)

num_classes = train_data.num_classes


Found 2780 images belonging to 3 classes.
Found 695 images belonging to 3 classes.


In [ ]:
class PatchExtractor(tf.keras.layers.Layer):
    def __init__(self, patch_size):
        super().__init__()
        self.patch_size = patch_size

    def call(self, images):
        batch_size = tf.shape(images)[0]
        patches = tf.image.extract_patches(
            images=images,
            sizes=[1, self.patch_size, self.patch_size, 1],
            strides=[1, self.patch_size, self.patch_size, 1],
            rates=[1, 1, 1, 1],
            padding='VALID'
        )
        patch_dims = patches.shape[-1]
        return tf.reshape(patches, [batch_size, -1, patch_dims])


In [ ]:
class PatchEncoder(tf.keras.layers.Layer):
    def __init__(self, num_patches, projection_dim):
        super().__init__()
        self.projection = Dense(projection_dim)
        self.position_embedding = self.add_weight(
            name="pos_embed",
            shape=(num_patches, projection_dim),
            initializer="random_normal",
            trainable=True
        )

    def call(self, patches):
        return self.projection(patches) + self.position_embedding


In [ ]:
# class PatchEncoder(tf.keras.layers.Layer):
#     def __init__(self, num_patches, projection_dim):
#         super().__init__()
#         self.projection = Dense(projection_dim)
#         self.position_embedding = self.add_weight("pos_embed", shape=[num_patches, projection_dim])

#     def call(self, patches):
#         return self.projection(patches) + self.position_embedding


In [ ]:
class TransformerBlock(tf.keras.layers.Layer):
    def __init__(self, projection_dim, num_heads, transformer_units, dropout=0.1):
        super().__init__()
        self.attention = tf.keras.layers.MultiHeadAttention(num_heads=num_heads, key_dim=projection_dim)
        self.dropout1 = Dropout(dropout)
        self.norm1 = LayerNormalization(epsilon=1e-6)
        self.dense_proj = tf.keras.Sequential([
            Dense(transformer_units[0], activation='relu'),
            Dense(transformer_units[1]),
        ])
        self.dropout2 = Dropout(dropout)
        self.norm2 = LayerNormalization(epsilon=1e-6)

    def call(self, inputs):
        x = self.attention(inputs, inputs)
        x = self.dropout1(x)
        x = self.norm1(inputs + x)
        proj = self.dense_proj(x)
        proj = self.dropout2(proj)
        return self.norm2(x + proj)


In [ ]:
# Parameters
patch_size = 16
projection_dim = 64
num_patches = (img_size // patch_size) ** 2
num_heads = 4
transformer_units = [projection_dim * 2, projection_dim]
gru_units = 64

# Input layer
inputs = Input(shape=(img_size, img_size, 3))

# Patch extraction and encoding
x = PatchExtractor(patch_size)(inputs)
x = PatchEncoder(num_patches, projection_dim)(x)

# Transformer
x = TransformerBlock(projection_dim, num_heads, transformer_units)(x)

# GRU layer
x = GRU(gru_units)(x)

# Output layer
x = Dropout(0.5)(x)
outputs = Dense(num_classes, activation='softmax')(x)

model = Model(inputs, outputs)
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
model.summary()


Model: "functional_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer_1 (InputLayer)      │ (None, 128, 128, 3)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ patch_extractor_1               │ (None, None, 768)      │             0 │
│ (PatchExtractor)                │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ patch_encoder_1 (PatchEncoder)  │ (None, 64, 64)         │        53,312 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ transformer_block               │ (None, 64, 64)         │        83,200 │
│ (TransformerBlock)              │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ gru (GRU)                       │ (None, 64)             │        24,960 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_3 (Dropout)             │ (None, 64)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_4 (Dense)                 │ (None, 3)              │           195 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 161,667 (631.51 KB)

 Trainable params: 161,667 (631.51 KB)

 Non-trainable params: 0 (0.00 B)

In [ ]:
history = model.fit(
    train_data,
    validation_data=val_data,
    epochs=10
)


/usr/local/lib/python3.11/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Epoch 1/10
87/87 ━━━━━━━━━━━━━━━━━━━━ 0s 272ms/step - accuracy: 0.3378 - loss: 1.4247

/usr/local/lib/python3.11/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


87/87 ━━━━━━━━━━━━━━━━━━━━ 37s 336ms/step - accuracy: 0.3378 - loss: 1.4231 - val_accuracy: 0.4158 - val_loss: 1.0554
Epoch 2/10
87/87 ━━━━━━━━━━━━━━━━━━━━ 29s 328ms/step - accuracy: 0.4218 - loss: 1.0744 - val_accuracy: 0.5453 - val_loss: 0.8854
Epoch 3/10
87/87 ━━━━━━━━━━━━━━━━━━━━ 31s 352ms/step - accuracy: 0.5445 - loss: 0.9058 - val_accuracy: 0.6331 - val_loss: 0.7711
Epoch 4/10
87/87 ━━━━━━━━━━━━━━━━━━━━ 29s 333ms/step - accuracy: 0.6222 - loss: 0.7968 - val_accuracy: 0.6532 - val_loss: 0.7257
Epoch 5/10
87/87 ━━━━━━━━━━━━━━━━━━━━ 29s 330ms/step - accuracy: 0.6262 - loss: 0.7577 - val_accuracy: 0.6576 - val_loss: 0.7297
Epoch 6/10
87/87 ━━━━━━━━━━━━━━━━━━━━ 29s 329ms/step - accuracy: 0.6493 - loss: 0.7277 - val_accuracy: 0.6604 - val_loss: 0.7107
Epoch 7/10
87/87 ━━━━━━━━━━━━━━━━━━━━ 28s 326ms/step - accuracy: 0.6592 - loss: 0.7487 - val_accuracy: 0.7007 - val_loss: 0.6476
Epoch 8/10
87/87 ━━━━━━━━━━━━━━━━━━━━ 29s 337ms/step - accuracy: 0.6791 - loss: 0.7059 - val_accuracy: 0.700

In [ ]:
loss, acc = model.evaluate(val_data)
print(f"Validation Accuracy: {acc * 100:.2f}%")

model.save("vit_gru_lung_disease_model.h5")


22/22 ━━━━━━━━━━━━━━━━━━━━ 4s 180ms/step - accuracy: 0.7432 - loss: 0.6110


Validation Accuracy: 74.24%
